In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append("../../")
from experiment import Exp

from dataloaders import data_set,data_dict
import torch
import yaml
import os

In [2]:
### Parameter for training

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

args = dotdict()   

args.to_save_path     = "home/lukasprobst/Documents/Bachelorarbeit/Run_logs"           
args.freq_save_path   = "home/lukasprobst/Documents/Bachelorarbeit/Freq_data"
args.window_save_path = "home/lukasprobst/Documents/Bachelorarbeit/Sliding_window"
args.root_path        = "home/lukasprobst/Documents/Bachelorarbeit/datasets"

args.drop_transition  = False
args.datanorm_type    = "standardization" # None ,"standardization", "minmax"


args.batch_size       = 256                                                     
args.shuffle          = True
args.drop_last        = False
args.train_vali_quote = 0.90                                           


# training setting 
args.train_epochs            = 150

args.learning_rate           = 0.001  
args.learning_rate_patience  = 7
args.learning_rate_factor    = 0.1


args.early_stop_patience     = 15

args.use_gpu                 = True if torch.cuda.is_available() else False
args.gpu                     = 0
args.use_multi_gpu           = False

args.optimizer               = "Adam"
args.criterion               = "CrossEntropy"

In [3]:
### Parameter for data

args.seed                             = 1


args.data_name                        =  "oppo"

args.wavelet_filtering                = False
args.wavelet_filtering_regularization = False
args.wavelet_filtering_finetuning     = False
args.wavelet_filtering_finetuning_percent = 0.5
args.wavelet_filtering_learnable      = False
args.wavelet_filtering_layernorm      = False

args.regulatization_tradeoff          = 0
args.number_wavelet_filtering         = 6


# Augmentation difference
args.difference          =  True 

# Augmentation filtering
args.filtering           =  True

# Augmentation  magnitude
args.magnitude           =  False
args.weighted_sampler    = False

# MIXUP
args.mixup_probability  = 0.5
args.mixup_alpha  = 0.5


# Random Augmentation
args.random_augmentation_prob = 0.5
args.random_augmentation_config = {"jitter":True,
                                   "moving_average":True,
                                   "magnitude_scaling":True,
                                   "magnitude_warp":True,
                                   "magnitude_shift":True,
                                   "time_warp":True,
                                   "window_warp":True,
                                   "window_slice":True,
                                   "random_sampling":True,
                                   "slope_adding":True
                                   }
random_augmentation_nr = 0
for key in args.random_augmentation_config.keys():
    if args.random_augmentation_config[key]:
        random_augmentation_nr = random_augmentation_nr+1
args.random_augmentation_nr = random_augmentation_nr
args.max_aug = 3

args.pos_select       = None
args.sensor_select    = None

args.representation_type = "time"
args.exp_mode            = "LOCV"
if args.data_name      ==  "skodar":
    args.exp_mode            = "SOCV"
    
config_file = open('../../configs/data.yaml', mode='r')
data_config = yaml.load(config_file, Loader=yaml.FullLoader)
config = data_config[args.data_name]

args.root_path       = os.path.join(args.root_path,config["filename"])
args.sampling_freq   = config["sampling_freq"]
args.num_classes     =  config["num_classes"]
window_seconds       = config["window_seconds"]
print
args.windowsize      =   int(window_seconds * args.sampling_freq) 
args.input_length    =  args.windowsize
# input information
args.c_in            =  config["num_channels"]
print(args.c_in )
if args.difference:
    args.c_in  = args.c_in * 2
    print(args.c_in )
if args.filtering:
    for col in config["sensors"]:
        if "acc" in col:
            args.c_in = args.c_in+1
            print(args.c_in )

if args.wavelet_filtering :
    
    if args.windowsize%2==1:
        N_ds = int(torch.log2(torch.tensor(args.windowsize-1)).floor()) - 2
    else:
        N_ds = int(torch.log2(torch.tensor(args.windowsize)).floor()) - 2

    args.f_in            =  args.number_wavelet_filtering*N_ds+1
else:
    args.f_in            =  1

77
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181


In [4]:
args.num_classes

18

In [5]:
### Parameter for the model

args.model_type = "shaspec"

args.decoder_type = "FC"
args.shared_encoder_type = "concatenated"

exp = Exp(args)

Use CPU


RuntimeError: Calculated padded input size per channel: (1 x 181). Kernel size: (5 x 1). Kernel size can't be greater than actual input size

In [ ]:
exp.model

model_builder(
  (model): ShaSpec(
    (specific_encoders): ModuleList(
      (0-5): 6 x SpecificEncoder(
        (conv1): Conv2d(1, 42, kernel_size=(5, 1), stride=(2, 1))
        (conv2): Conv2d(42, 42, kernel_size=(5, 1), stride=(2, 1))
        (conv3): Conv2d(42, 42, kernel_size=(5, 1), stride=(2, 1))
        (conv4): Conv2d(42, 42, kernel_size=(5, 1), stride=(2, 1))
        (activation): ReLU()
        (sa): SelfAttention(
          (query): Conv1d(42, 5, kernel_size=(1,), stride=(1,), bias=False)
          (key): Conv1d(42, 5, kernel_size=(1,), stride=(1,), bias=False)
          (value): Conv1d(42, 42, kernel_size=(1,), stride=(1,), bias=False)
        )
        (fc_fusion): Linear(in_features=210, out_features=84, bias=True)
      )
    )
    (shared_encoder): SharedEncoder(
      (conv1): Conv2d(1, 42, kernel_size=(5, 1), stride=(2, 1))
      (conv2): Conv2d(42, 42, kernel_size=(5, 1), stride=(2, 1))
      (conv3): Conv2d(42, 42, kernel_size=(5, 1), stride=(2, 1))
      (conv4):

In [ ]:
exp.train()

NameError: name 'exp' is not defined